# NLP.F2501 Course Project 1 (Word embeddings and RNNs)

Nevin Helfenstein

# Introduction

In this notebook, I present my solution to the CommonsenseQA task. I'll train a model using word embeddings, RNNs, and other NLP techniques to achieve the best possible performance.

## Dataset Description

The CommonsenseQA dataset [(Talmor et al., 2019)](https://aclanthology.org/N19-1421/) contains 12,247 multiple-choice questions specifically designed to test commonsense reasoning. Unlike standard QA tasks, these questions require prior knowledge about how concepts relate in the real world.

Questions were created by extracting related concepts from ConceptNet and having crowd-workers author questions that require distinguishing between them. This methodology produced challenging questions that often cannot be answered through simple pattern matching.

The best baseline in the original paper (BERT-large) achieved only 56% accuracy compared to human performance of 89%, shwoing the difficulty of encoding human-like commonsense reasoning.

# Setup

First we import all the needed libraries

In [ ]:
import torch
import nltk
import wandb
import logging
import re

import pandas as pd
import seaborn as sns
from wordcloud import WordCloud

import torch.nn as nn
import numpy as np

import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from torch.utils.data import Dataset, DataLoader
import gensim.downloader as api
from matplotlib_venn import venn2

from datasets import load_dataset
from huggingface_hub import hf_hub_download

from collections import Counter
from datetime import datetime


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/maiko/.netrc
wandb: Currently logged in as: maikotrede (maikotrede-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### WandDB

We set up the configuration to W&B for later use

In [ ]:
#TODO: Correct configuration for the new wanddb project

wandb.login(key="")
wandb_logger = WandbLogger(project="experiment-tracking")

def init_wandb_run(project_name, run_name, config_dict):
    run = wandb.init(
        project=project_name,
        name=run_name,
        config=config_dict,
    )
    return run


#### Example Run
experiment_1_config = {
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 32,
    "optimizer": "adam"
}
run1 = init_wandb_run("experiment-tracking", "experiment_1", experiment_1_config)


### Fixed variables

We set the random seed for all the necessary configurations  to ensure reproducibility

In [ ]:
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Tokenizer and embedding model

I've selected GloVe embeddings for this project for the following reasons:

* Merges global statistics with local context data
* Strong performance on semantic similarity benchmarks relevant to commonsense reasoning
* 300-dimensional vectors balance representation ability with efficiency
* Strongly established in NLP community with good documentation
* Effectively models conceptual relationships for answering commonsense questions

Download tokenizer files

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

Load GloVe embeddings using gensim downloader (300d Wikipedia + Gigaword model)

In [ ]:
glove_model = api.load('glove-wiki-gigaword-300')

Create a function to get embeddings for words (Return zero vector for OOV words)

In [ ]:
def get_glove_embedding(word):
    try:
        return glove_model[word]
    except KeyError:
        return np.zeros(300)

### Data Splits

The data is available on Hugging Face: [Data](https://huggingface.co/datasets/tau/commonsense_qa).
Only the train and validation splits have an answer key, because of that we will use our own dataset splits.
We use all of the datasamples as the train set except for the last 1000 which we set as the validation set. The original validation set is set as the test set.

In [ ]:
train = load_dataset("tau/commonsense_qa", split="train[:-1000]")
valid = load_dataset("tau/commonsense_qa", split="train[-1000:]")
test = load_dataset("tau/commonsense_qa", split="validation")

print(len(train), len(valid), len(test))

## Data exploration

We convert to DataFrames for easier analysis

In [ ]:
train_df = pd.DataFrame(train)
valid_df = pd.DataFrame(valid)
test_df = pd.DataFrame(test)

We check basic statistics such as average question and choice lenght for each DataFrame

In [ ]:
def analyze_dataset(df, name):
    """Analyze basic statistics of a dataset split"""
    print(f"=== {name} Set Statistics ===")
    print(f"Number of examples: {len(df)}")
    
    # Question statistics
    df['question_tokens'] = df['question'].apply(lambda x: word_tokenize(x))
    df['question_length'] = df['question_tokens'].apply(len)
    
    # Answer choices statistics
    df['choices_length'] = df['choices'].apply(lambda x: [len(word_tokenize(choice['text'])) for choice in x])
    df['avg_choice_length'] = df['choices_length'].apply(np.mean)
    
    print(f"Average question length: {df['question_length'].mean():.2f} tokens")
    print(f"Average answer choice length: {df['avg_choice_length'].mean():.2f} tokens")
    print(f"Min/Max question length: {df['question_length'].min()}/{df['question_length'].max()} tokens")
    
    # Find correct answer position
    df['correct_answer_idx'] = df.apply(lambda row: next((i for i, choice in enumerate(row['choices']) 
                                                         if choice['label'] == row['answerKey']), -1), axis=1)
    
    return df

train_df = analyze_dataset(train_df, "Training")
valid_df = analyze_dataset(valid_df, "Validation")
test_df = analyze_dataset(test_df, "Test")

We plot the question lenght distribution

In [ ]:
plt.figure(figsize=(12, 6))

sns.histplot(data=train_df, x='question_length', kde=True, label='Train', alpha=0.6)
sns.histplot(data=valid_df, x='question_length', kde=True, label='Validation', alpha=0.6)
sns.histplot(data=test_df, x='question_length', kde=True, label='Test', alpha=0.6)

plt.title('Distribution of Question Lengths')
plt.xlabel('Number of tokens in question')
plt.ylabel('Count')
plt.legend()
plt.savefig('question_length_distribution.png')
plt.close()

We plot the answer length distribution

In [ ]:
plt.figure(figsize=(12, 6))

sns.histplot(data=train_df, x='avg_choice_length', kde=True, label='Train', alpha=0.6)
sns.histplot(data=valid_df, x='avg_choice_length', kde=True, label='Validation', alpha=0.6)
sns.histplot(data=test_df, x='avg_choice_length', kde=True, label='Test', alpha=0.6)

plt.title('Distribution of Answer Choice Lengths')
plt.xlabel('Average number of tokens in answer choices')
plt.ylabel('Count')
plt.legend()
plt.savefig('answer_length_distribution.png')
plt.close()

Count correct answer keys

In [ ]:
plt.figure(figsize=(10, 6))

train_pos_counts = Counter(train_df['correct_answer_idx'])
valid_pos_counts = Counter(valid_df['correct_answer_idx'])
test_pos_counts = Counter(test_df['correct_answer_idx'])

Convert that to percentage

In [ ]:
train_pos_percent = {k: v/len(train_df)*100 for k, v in train_pos_counts.items()}
valid_pos_percent = {k: v/len(valid_df)*100 for k, v in valid_pos_counts.items()}
test_pos_percent = {k: v/len(test_df)*100 for k, v in test_pos_counts.items()}

Create a DataFrame again for better plotting

In [ ]:
pos_df = pd.DataFrame({
    'Train': [train_pos_percent.get(i, 0) for i in range(5)],
    'Validation': [valid_pos_percent.get(i, 0) for i in range(5)],
    'Test': [test_pos_percent.get(i, 0) for i in range(5)]
}, index=['A', 'B', 'C', 'D', 'E'])

pos_df.plot(kind='bar', figsize=(10, 6))
plt.title('Distribution of Correct Answer Positions')
plt.xlabel('Answer Position')
plt.ylabel('Percentage (%)')
plt.legend()
plt.savefig('answer_position_distribution.png')
plt.close()

Create word clouds for questions

In [ ]:
def create_wordcloud(texts, title, filename):
    """Create and save wordcloud from a list of texts"""
    plt.figure(figsize=(12, 8))
    
    all_text = ' '.join(texts)
    
    wordcloud = WordCloud(width=800, height=400, 
                         background_color='white',
                         max_words=100, 
                         contour_width=3).generate(all_text)
    
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=16)
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

create_wordcloud(train_df['question'].tolist(), 'Word Cloud - Training Questions', 'train_question_wordcloud.png')

Analyze question types (what, how, why, etc.)

In [ ]:
def get_question_type(question):
    """Extract the question word from a question"""
    question = question.lower().strip()
    question_words = ['what', 'which', 'who', 'how', 'why', 'when', 'where']
    
    for word in question_words:
        if question.startswith(word) or f" {word} " in question:
            return word
    
    return 'other'

train_df['question_type'] = train_df['question'].apply(get_question_type)
valid_df['question_type'] = valid_df['question'].apply(get_question_type)
test_df['question_type'] = test_df['question'].apply(get_question_type)

plt.figure(figsize=(12, 6))

Count question types

In [ ]:
train_type_counts = Counter(train_df['question_type'])
valid_type_counts = Counter(valid_df['question_type'])
test_type_counts = Counter(test_df['question_type'])

Converte to percentage

In [ ]:
train_type_percent = {k: v/len(train_df)*100 for k, v in train_type_counts.items()}
valid_type_percent = {k: v/len(valid_df)*100 for k, v in valid_type_counts.items()}
test_type_percent = {k: v/len(test_df)*100 for k, v in test_type_counts.items()}

Plot the question type distribution

In [ ]:
common_types = set(list(train_type_counts.keys()) + list(valid_type_counts.keys()) + list(test_type_counts.keys()))
type_df = pd.DataFrame({
    'Train': [train_type_percent.get(t, 0) for t in common_types],
    'Validation': [valid_type_percent.get(t, 0) for t in common_types],
    'Test': [test_type_percent.get(t, 0) for t in common_types]
}, index=common_types)

type_df.plot(kind='bar', figsize=(12, 6))
plt.title('Distribution of Question Types')
plt.xlabel('Question Type')
plt.ylabel('Percentage (%)')
plt.legend()
plt.savefig('question_type_distribution.png')
plt.close()

POS (Part-of-Speach) Tag analysis for questions

In [ ]:
def analyze_pos_tags(texts, n=10):
    """Analyze the most common POS tags in a list of texts"""
    all_pos = []
    for text in texts:
        tokens = word_tokenize(text)
        tags = pos_tag(tokens)
        all_pos.extend([tag for _, tag in tags])
    
    return Counter(all_pos).most_common(n)

# Get most common POS tags in train questions
train_pos = analyze_pos_tags(train_df['question'].tolist())

plt.figure(figsize=(12, 6))
pos_df = pd.DataFrame(train_pos, columns=['POS Tag', 'Count'])
sns.barplot(x='POS Tag', y='Count', data=pos_df)
plt.title('Most Common POS Tags in Training Questions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('pos_tag_distribution.png')
plt.close()

Analyze vocabulary overlap between train and test

In [ ]:
def get_vocab(texts):
    """Get vocabulary from a list of texts"""
    vocab = set()
    for text in texts:
        tokens = word_tokenize(text.lower())
        vocab.update(tokens)
    return vocab

train_vocab = get_vocab(train_df['question'].tolist())
test_vocab = get_vocab(test_df['question'].tolist())

Calculate overlap and plot venn diagramm

In [ ]:
overlap = len(train_vocab.intersection(test_vocab))
train_only = len(train_vocab - test_vocab)
test_only = len(test_vocab - train_vocab)

plt.figure(figsize=(10, 6))

venn2(subsets=(train_only, test_only, overlap), 
      set_labels=('Train Vocabulary', 'Test Vocabulary'))
plt.title('Vocabulary Overlap Between Train and Test Sets')
plt.savefig('vocab_overlap.png')
plt.close()

print(f"Train vocabulary size: {len(train_vocab)}")
print(f"Test vocabulary size: {len(test_vocab)}")
print(f"Vocabulary overlap: {overlap} words ({overlap/len(train_vocab)*100:.2f}% of train vocab)")

# Preprocessing

README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

# Model

# Training

# Evaluation

# Interpretation